In [27]:
import pandas as pd
import numpy as np
from pycaret.regression import *

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error

In [28]:
df = pd.read_csv('encoded_df.csv')

In [29]:
df1 = df.copy()

In [30]:
df1.head()

,Wk,Winner,Pts,Loser,Pts.1,winners,losers,home_rank,away_rank,home_team_code,away_team_code,home_results,win_rank,lose_rank,day_code,hours,null,half,quart,three_quart
0,1,Florida Atlantic,43,Charlotte,13,Florida Atlantic,Charlotte,99,99,32,37,0,99,99,5,19,1,0,0,0
1,1,Florida State,47,Duquesne,7,Florida State,Duquesne,99,99,34,49,0,99,99,5,17,1,0,0,0
2,1,Illinois,38,Wyoming,6,Illinois,Wyoming,99,99,42,224,0,99,99,5,16,1,0,0,0
3,1,Nevada,23,New Mexico State,12,Nevada,New Mexico State,99,99,76,122,1,99,99,5,22,1,0,0,0
4,1,Nevada-Las Vegas,52,Idaho State,21,Nevada-Las Vegas,Idaho State,99,99,74,77,0,99,99,5,15,0,1,0,0


In [31]:
df1[df1['Pts'] == 0]

,Wk,Winner,Pts,Loser,Pts.1,winners,losers,home_rank,away_rank,home_team_code,away_team_code,home_results,win_rank,lose_rank,day_code,hours,null,half,quart,three_quart


In [32]:
df.shape

(778, 20)

In [33]:
train = df1.iloc[:714]
test = df1.iloc[714:]

In [44]:
X_train = train.drop(columns=['Winner', 'Loser', 'winners', 'losers',
                              'null', 'half', 'quart', 'three_quart', 
                               'Pts.1', 'hours', 'day_code'])

X_test = test.drop(columns=['Winner', 'Loser', 'winners', 'losers',
                              'null', 'half', 'quart', 'three_quart', 
                              'Pts.1', 'hours', 'day_code'])

In [45]:
X_test

,Wk,Pts,home_rank,away_rank,home_team_code,away_team_code,home_results,win_rank,lose_rank
714,13,32,99,99,11,143,1,99,99
715,13,31,99,99,49,54,1,99,99
716,13,29,99,99,80,109,1,99,99
717,13,12,99,99,20,220,1,99,99
718,13,59,21,99,116,175,0,21,99
719,13,34,99,99,75,162,1,99,99
720,13,48,99,99,117,169,0,99,99
721,13,24,99,99,0,45,0,99,99
722,13,34,8,99,2,17,0,8,99
723,13,27,99,99,4,147,0,99,99


In [46]:
y_train = train['Pts.1']
y_test = test['Pts.1']

In [47]:
X_train

,Wk,Pts,home_rank,away_rank,home_team_code,away_team_code,home_results,win_rank,lose_rank
0,1,43,99,99,32,37,0,99,99
1,1,47,99,99,34,49,0,99,99
2,1,38,99,99,42,224,0,99,99
3,1,23,99,99,76,122,1,99,99
4,1,52,99,99,74,77,0,99,99
...,...,...,...,...,...,...,...,...,...
709,12,28,99,99,126,11,0,99,99
710,12,23,99,99,127,145,0,99,99
711,12,45,99,99,128,155,0,99,99
712,12,14,99,99,26,224,1,99,99


In [48]:
# # Ignoring features with high null values 

# demo = setup(data = train, target = 'Pts', 
#                    transformation= True, transformation_method = 'yeo-johnson', 
#                    transform_target = True, remove_outliers= True,
#                    remove_multicollinearity = True, imputation_type='iterative',
#                    ignore_low_variance = True, combine_rare_levels = True, fold_shuffle=True,) 



In [49]:
# best = compare_models()

In [50]:
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor

In [51]:
clf = Ridge(alpha=.001)
clf.fit(X_train, y_train)

Ridge(alpha=0.001)

In [52]:
regr = RandomForestRegressor(max_depth=2, random_state=0)
regr.fit(X_train, y_train)

RandomForestRegressor(max_depth=2, random_state=0)

In [53]:
test_pred = clf.predict(X_test)

print('RMSE: '+str(np.sqrt(mean_squared_error(y_test, test_pred))))
print('R Squared: '+str(r2_score(y_test, test_pred)))
print('MSE TEST: '+str(mean_squared_error(y_test, test_pred)))
print('MAE TEST: '+str(mean_absolute_error(y_test, test_pred)))
print('MAPE: '+str(mean_absolute_percentage_error(y_test, test_pred)))

RMSE: 10.090026617128313
R Squared: 0.05400637976854328
MSE TEST: 101.80863713435781
MAE TEST: 8.301830867128778
MAPE: 4075855563100421.0


In [54]:
# RMSE: 12.98527052295243
# R Squared: 0.003458085844145864
# MSE TEST: 168.61725055425725
# MAE TEST: 10.68224945264794
# MAPE: 0.3613084327156201

# alpha=0 MAPE: 0.36128976519731065

In [55]:
data = {'y_test': y_test, 'y_pred': test_pred}
new_df = pd.DataFrame(data)
diff = (new_df['y_test'] - new_df['y_pred']).abs()
new_df['diff'] = diff
new_df = new_df.sort_values(by='diff', ascending=True)
new_df

,y_test,y_pred,diff
776,20,19.810082,0.189918
760,20,19.615081,0.384919
724,27,27.558157,0.558157
748,21,21.827424,0.827424
733,17,18.915218,1.915218
725,17,18.996958,1.996958
726,17,19.129355,2.129355
727,17,19.344263,2.344263
714,18,20.491811,2.491811
716,23,20.313623,2.686377


In [56]:
predz = new_df['y_pred']
new_pred = []

for i in predz:
    new_pred.append(round(i))

In [57]:
new_pred.sort()

In [58]:
new_df['y_pred'].value_counts()

19.810082    1
19.615081    1
22.633888    1
17.476455    1
17.616620    1
18.043217    1
19.604245    1
22.450309    1
22.333487    1
22.848244    1
19.148802    1
24.546231    1
20.875046    1
24.013764    1
18.111913    1
18.630505    1
16.208271    1
20.234680    1
20.250305    1
20.244895    1
20.374713    1
24.909397    1
21.988505    1
16.191330    1
23.193468    1
19.721205    1
28.137642    1
18.524402    1
20.419746    1
22.870965    1
21.007124    1
17.982144    1
24.097533    1
23.892300    1
16.808086    1
27.558157    1
21.827424    1
18.915218    1
18.996958    1
19.129355    1
19.344263    1
20.491811    1
20.313623    1
18.867829    1
16.916209    1
16.994933    1
20.992566    1
16.084852    1
20.526692    1
16.863303    1
20.454611    1
18.445788    1
17.178291    1
12.948924    1
21.288498    1
17.119611    1
18.606783    1
31.987551    1
16.259371    1
15.369567    1
20.635277    1
16.770570    1
16.837137    1
23.205659    1
Name: y_pred, dtype: int64

In [59]:
new_df['diff'].mean()

8.301830867128778

- michigan awaycode = 110 rank3 WIN
- ohio state homecode = 83 rank2
- 1


- oregon awaycode = 148 rank 10 WIN
- oregon state homecode = 89 rnak 22
- 1


- notre dame awaycode = 142 rank 13
- usc homecode = 100 rank 5 WIN
-  0


- tulene awaycode = 200 rank 19 WIN
- cincinati  homecode = 22 rank 21
- 1


- baylor homecode = 19 rank 99
- texas awaycode=189 rank 24 WIN
- 0


- ms state awaycode=115 rank99
- ole miss homecode=67 rank 20 WIN
- 0


In [60]:
ba = [[14, 31, 99, 24, 19, 189, 1, 24, 99]]
bay = pd.DataFrame(ba, columns=['Wk', 'Pts', 'home_rank', 'away_rank',
                                'home_team_code', 'away_team_code', 
                                'home_results', 'win_rank', 
                                'lose_rank'])
bay

# oh = [[14, 2, 3, 83, 110, 1, 3, 2]]
# osu = pd.DataFrame(oh, columns=['Wk', 'home_rank', 'away_rank',
#                                 'home_team_code', 'away_team_code', 
#                                 'home_results', 'win_rank', 
#                                 'lose_rank'])
# osu

# og = [[14, 31, 22, 10, 89, 148, 1, 10, 22, 5]]
# uo = pd.DataFrame(og, columns=['Wk', 'Pts', 'home_rank', 'away_rank',
#                                 'home_team_code', 'away_team_code', 
#                                 'home_results', 'win_rank', 
#                                 'lose_rank', 'day_code'])
# uo


# sc = [[14, 33, 5, 13, 100, 142, 0, 5, 13, 5]]
# usc = pd.DataFrame(sc, columns=['Wk', 'Pts', 'home_rank', 'away_rank',
#                                 'home_team_code', 'away_team_code', 
#                                 'home_results', 'win_rank', 
#                                 'lose_rank', 'day_code'])
# usc

# tu = [[14, 29, 21, 19, 22, 200, 1, 19, 21, 5]]
# tul = pd.DataFrame(tu, columns=['Wk', 'Pts', 'home_rank', 'away_rank',
#                                 'home_team_code', 'away_team_code', 
#                                 'home_results', 'win_rank', 
#                                 'lose_rank', 'day_code'])
# tul


# la = [[14, 2, 3, 83, 110, 1, 3, 2]]
# lsu = pd.DataFrame(la, columns=['Wk', 'home_rank', 'away_rank',
#                                 'home_team_code', 'away_team_code', 
#                                 'home_results', 'win_rank', 
#                                 'lose_rank'])
# lsu


,Wk,Pts,home_rank,away_rank,home_team_code,away_team_code,home_results,win_rank,lose_rank
0,14,31,99,24,19,189,1,24,99


In [61]:
clf.predict(bay)


array([20.33055417])

In [1]:
# Texas 31.24
# Baylor 20.330

# results: Texas 38 Baylor 27


# ----

# ohio state 30.68
# michigan 26.316

# oddshark 
# michigan 40.9 ohioStae 45.7

# results: michigan 45 ohioStae 23


# ------

# Oregon 30.783
# Oregon State 24.936

# oddshark
# oregon 43.2 - oregon state 35.7

# results: oregon 34 oregon state 38


# ------

# USC 32.981
# Notre Dame 23.154

# oddshark
# notre dame 36.6 - USC 38.0

# results: notre dame 27 USC 38



# ----


# oddshark 
# lsu 35.0 - Texas A&M 28.4

# results: lsu 23 Texas A&M 38


# Results: tulane 27 - cincinnati 24


# Prediction: tulane 29.4244 - Cinncinati 24.165

# oddshark (diff than my prediction): Tulane 33.6 - Cincinnati 38.6

In [87]:
from sklearn.linear_model import OrthogonalMatchingPursuit

reg = OrthogonalMatchingPursuit(fit_intercept=True, n_nonzero_coefs=8,
                          normalize=False, precompute='auto', tol=None)
# reg.fit(X_train, y_train)

In [88]:
reg.fit(X_train, y_train)

OrthogonalMatchingPursuit(fit_intercept=True, n_nonzero_coefs=8,
                          normalize=False, precompute='auto', tol=None)

In [89]:
test_pred = reg.predict(X_test)

print('RMSE: '+str(np.sqrt(mean_squared_error(y_test, test_pred))))
print('R Squared: '+str(r2_score(y_test, test_pred)))
print('MSE TEST: '+str(mean_squared_error(y_test, test_pred)))
print('MAE TEST: '+str(mean_absolute_error(y_test, test_pred)))
print('MAPE: '+str(mean_absolute_percentage_error(y_test, test_pred)))

RMSE: 12.936093507505246
R Squared: 0.021070137731252325
MSE TEST: 167.3425152349194
MAE TEST: 10.571476067349222
MAPE: 0.3621416133095708


In [92]:
data = {'y_test': y_test, 'y_pred': test_pred}
new_df = pd.DataFrame(data)
diff = (new_df['y_test'] - new_df['y_pred']).abs()
new_df['diff'] = diff
new_df = new_df.sort_values(by='diff', ascending=True)
new_df

,y_test,y_pred,diff
715,31,30.421945,0.578055
737,31,31.754156,0.754156
754,28,28.797381,0.797381
756,31,31.941905,0.941905
770,34,32.786101,1.213899
...,...,...,...
718,59,36.721875,22.278125
749,56,32.465082,23.534918
763,55,29.700320,25.299680
746,59,31.965283,27.034717


In [94]:
predz = new_df['y_pred']
new_pred = []

for i in predz:
    new_pred.append(round(i))

In [91]:
reg.predict(osu)

array([30.8160472])